## Inspired from

https://github.com/SakuraSa/Leetcode_CodeDownloader/blob/cba23e3ec85b24e14fdf856e0e7eefb2c95644eb/Leetcode.py
    
https://github.com/tedye/leetcode/blob/975d7e3b8cb9b6be9e80e07febf4bcf6414acd46/tools/leetcode_submission_crawler.py
    
    
Although lot of code changes had to be done otherwise to accomplish what I needed.


In [6]:
import os
import re
import requests
import datetime
from bs4 import BeautifulSoup

In [22]:
import json

In [ ]:
import pickle

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import re

In [142]:
from selenium.webdriver.common.action_chains import ActionChains

In [89]:
from typing import List, Dict, Tuple

In [98]:
from ipypb import track

In [99]:
from time import sleep

In [100]:
for i in track(range(10)):
    sleep(0.1)

 [████████████████████████████████████████████████████████████] 10/10 [00:01<00:00, 0.10s/it]

# Login

In [144]:
CODE_DRIVER = webdriver.Chrome("./chromedriver")


In [183]:


USERNAME  = 'ENTER_YOUR_USERNAME_HERE'
PASSWORD = 'ENTER_YOUR_PASSWORD_HERE'

In [146]:
CODE_DRIVER.implicitly_wait(8)

# CODE_DRIVER.get("https://leetcode.com/accounts/logout")

CODE_DRIVER.get("https://leetcode.com/accounts/login/")

CODE_DRIVER.find_element_by_xpath('// *[ @ id = "id_login"]').send_keys(USERNAME)
CODE_DRIVER.find_element_by_xpath('// *[ @ id = "id_password"]').send_keys(PASSWORD)
CODE_DRIVER.find_element_by_xpath('// *[ @ id = "id_password"]').send_keys(Keys.ENTER)

# Get fb questions

In [123]:
CODE_DRIVER.get("https://leetcode.com/company/facebook/")

In [111]:
table = CODE_DRIVER.find_element_by_class_name('reactable-data')

title_href = {}

for row in table.find_elements_by_tag_name("tr"):
    title = row.find_element_by_tag_name("a").text
    href = row.find_element_by_tag_name("a").get_attribute("href")
    title_href[title] = href

In [147]:
len(title_href)

336

In [41]:
with open('facebook_problems_list.json', 'w') as fp:
    json.dump(title_href, fp, indent=4)

# Scrape the information we need

problem name, likes, dislikes, companies, difficulty, tags, lists

```
- lists : /html/body/div[8]/div/div/div/div[2]/div/div/div/div  (search for div insides pick text from ones which have class that contains "favorite-selected" and ignore "favorite-unselected")

- problem name : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[1]  (get text)

- likes : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[1]  (.text)

- dislikes : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[2]  (.text)

- companies : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[6]/div[2]/div  (go through a tags inside, for each a tag get its text (woud be like "Facebook | 153" -- now deconstruct this into a dictionary)

- difficulty : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/div  (.text)

- tags : //*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[7]/div[2] (search through a elements inside, get their text and append to a list)

```

In [42]:
# try for one problem 
wait = WebDriverWait(CODE_DRIVER, 10)

CODE_DRIVER.get('https://leetcode.com/problems/task-scheduler')

lists = []
problem_name = None
likes = None
dislikes = None
companies = dict()
difficulty = None
tags = []

In [43]:
problem_name = CODE_DRIVER.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[1]').text

In [45]:
likes = CODE_DRIVER.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[1]').text
dislikes = CODE_DRIVER.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[2]').text

In [48]:
likes, dislikes = int(likes), int(dislikes)

In [50]:
difficulty = CODE_DRIVER.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/div').text

In [74]:
lists_data_parent = CODE_DRIVER.find_element_by_xpath('/html/body/div[7]/div/div/div/div[2]/div/div/div/div')

for list_data_div in lists_data_parent.find_elements_by_tag_name('div'):
    if list_data_div.get_attribute("class").startswith('favorite-selected'):
        text_div = list_data_div.find_element_by_tag_name('div').find_element_by_tag_name('div')
        lists.append(text_div.get_attribute('innerHTML'))

In [75]:
lists

['CouldNotFigureOut']

In [84]:
companies_info = CODE_DRIVER.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[6]/div[2]/div')

for company_anchor_tags in companies_info.find_elements_by_tag_name('a'):
    company_text = company_anchor_tags.text.replace('\n','')
    company_data = company_text.split('|')
    companies[company_data[0]] = int(company_data[1])
    print(f'company text : {company_text}')

company text : Facebook|20
company text : Microsoft|6
company text : Amazon|2
company text : Bloomberg|2


In [85]:
companies

{'Facebook': 20, 'Microsoft': 6, 'Amazon': 2, 'Bloomberg': 2}

In [174]:
class LeetcodeProblemInformationScraper:
    
    def __init__(self, problem_name, problem_href, CODE_DRIVER):
        self.problem_info_dict = dict()
        self.problem_info_dict['name'] = problem_name
        self.problem_info_dict['href'] = problem_href
        self.browser = CODE_DRIVER
        self.browser.implicitly_wait(10)
        # visit the URL
        # try for one problem 
        wait = WebDriverWait(self.browser, 15)
        self.browser.get(problem_href)
    
    def find_name(self):
        xpath = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[1]'
        problem_text = self.browser.find_element_by_xpath(xpath).text
        self.problem_info_dict['full_name'] = problem_text
    
    def find_likes_dislikes(self):
        xpath_likes = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[1]'
        xpath_dislikes = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[2]'
        self.problem_info_dict['likes'] = int(self.browser.find_element_by_xpath(xpath_likes).text)
        self.problem_info_dict['dislikes'] = int(self.browser.find_element_by_xpath(xpath_dislikes).text)
    
    def find_difficulty(self):
        xpath_difficulty = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/div'
        self.problem_info_dict['difficulty'] = self.browser.find_element_by_xpath(xpath_difficulty).text
    
    def find_lists(self):
        lists = []
        try:
            wait = WebDriverWait(self.browser, 10)
            lists_header_xpath = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/button[3]'
            lists_elt = wait.until(EC.element_to_be_clickable((By.XPATH, lists_header_xpath)))
            hover = ActionChains(self.browser).move_to_element(lists_elt)
            hover.perform()
            time.sleep(0.5)

            lists_parent_xpath = '/html/body/div[7]/div/div/div/div[2]/div/div/div/div'
            lists_data_parent = wait.until(EC.presence_of_element_located((By.XPATH, lists_parent_xpath)))
            time.sleep(0.25)
            # find divs inside, and choose labels of those who have been selected
            for list_data_div in lists_data_parent.find_elements_by_tag_name('div'):
                if list_data_div.get_attribute("class").startswith('favorite-selected'):
                    text_div = list_data_div.find_element_by_tag_name('div').find_element_by_tag_name('div')
                    lists.append(text_div.get_attribute('innerHTML'))
            lists.sort()
        except Exception as e:
            self.problem_info_dict['error_finding_list'] = True
            print(f"question : {self.problem_info_dict['name']} | Problem with lists | exception : {e}")
        self.problem_info_dict['lists'] = lists
    
    def find_company_frequencies(self):
        company_info_dict = dict()
        company_data = None
        try:
            wait = WebDriverWait(self.browser, 10)
                                   
            company_header_xpath = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[6]/div[1]'
            elt = wait.until(EC.element_to_be_clickable((By.XPATH, company_header_xpath)))
            elt.click()
            time.sleep(0.25)
            company_info_xpath = '//*[@id="app"]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div[6]/div[2]/div'
            # companies_info = self.browser.find_element_by_xpath(company_info_xpath)
            companies_info = wait.until(EC.presence_of_element_located((By.XPATH, company_info_xpath)))
            # find anchor tags inside, get their text and format it better
            company_anchor_tags_list = wait.until(lambda d:companies_info.find_elements_by_tag_name('a')) 
            time.sleep(0.5)
            for company_anchor_tags in companies_info.find_elements_by_tag_name('a'):
                company_text = company_anchor_tags.text.replace('\n','')
                company_data = company_text.split('|')
                company_info_dict[company_data[0]] = int(company_data[1])
        except Exception as e:
            self.problem_info_dict['error_finding_company_frequency'] = True
            print(f"question : {self.problem_info_dict['name']} | Problem with company frequency | company data : {company_data} | exception : {e}")
        self.problem_info_dict['companies'] = company_info_dict
    
    def get_all_info(self) -> Dict:
        state = 'find name'
        try:
            self.find_name()
            state = 'find likes and dislikes'
            self.find_likes_dislikes()
            state = 'find difficulty'
            self.find_difficulty()
            state = 'find lists'
            self.find_lists()
            state = 'find company frequency'
            self.find_company_frequencies()
            return self.problem_info_dict
        except Exception as e:
            print(f"Question : {self.problem_info_dict['name']}, State : {state}, Exception : {e}")
            return None

# Scraping for each fb problem

In [177]:
fb_problems_details = []

In [180]:
len(fb_problems_details)

336

In [179]:
# Scraping for each...
for title, href in track(title_href.items()):
    current_problem = LeetcodeProblemInformationScraper(title, href, CODE_DRIVER)
    information_dict = current_problem.get_all_info()
    if information_dict is not None:
        fb_problems_details.append(information_dict)


 [████████████████████████████████████████████████████████████] 336/336 [01:18:25<00:15, 14.00s/it]

question : Longest Continuous Increasing Subsequence | Problem with lists | exception : Message: 

question : Best Time to Buy and Sell Stock | Problem with lists | exception : Message: 

question : Binary Tree Vertical Order Traversal | Problem with lists | exception : Message: 

question : Binary Tree Paths | Problem with lists | exception : Message: 

question : Binary Search Tree Iterator | Problem with company frequency | company data : None | exception : Message: element click intercepted: Element <div class="css-1819af6-baseHeaderStyle e5i1odf0">...</div> is not clickable at point (214, 637). Other element would receive the click: <div id="initial-loading">...</div>
  (Session info: chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Mac OS X 10.15.6 x86_64)

question : Reverse Words in a String | Problem with company frequency | company data : [''] | exception : list index out of range
q

In [172]:
len(fb_problems_details)

9

In [182]:
with open('facebook_problems_details.json', 'w') as fp:
    json.dump(fb_problems_details, fp, indent=4)

In [181]:
fb_problems_details

[{'name': 'Task Scheduler',
  'href': 'https://leetcode.com/problems/task-scheduler',
  'full_name': '621. Task Scheduler',
  'likes': 3475,
  'dislikes': 704,
  'difficulty': 'Medium',
  'lists': [],
  'companies': {'Facebook': 20, 'Microsoft': 6, 'Amazon': 2, 'Bloomberg': 2}},
 {'name': 'Merge Intervals',
  'href': 'https://leetcode.com/problems/merge-intervals',
  'full_name': '56. Merge Intervals',
  'likes': 4433,
  'dislikes': 302,
  'difficulty': 'Medium',
  'lists': ['Done_made_a_silly_though', 'RecommendedQuestions2Solv'],
  'companies': {'Facebook': 40,
   'Amazon': 26,
   'Bloomberg': 10,
   'Oracle': 9,
   'Microsoft': 8,
   'Apple': 8,
   'Paypal': 7,
   'Google': 7,
   'Uber': 7,
   'Cisco': 4,
   'LinkedIn': 4,
   'Yandex': 4,
   'ByteDance': 4,
   'Palantir Technologies': 3,
   'eBay': 3,
   'VMware': 3,
   'Adobe': 2,
   'Goldman Sachs': 2,
   'Netflix': 2,
   'DoorDash': 2}},
 {'name': 'Intersection of Two Arrays II',
  'href': 'https://leetcode.com/problems/intersect